In [178]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [179]:
def feedback(task, a1, a2, count):
    """Gives feedback based on numbers given from tracking"""

    #Pushup Feedback
    if task == 1:
        print("Pushup Results")
        print("\tYou did {} complete pushups".format(count))
        print("\tYour back average bend was {} degree".format(a1))
        print("\tYour elbow bend minimum was {} degree".format(a2))
        if a1 < 160:
            print("\tTry to keep your back more straight a good way is to keep your but down")
            print("\tThe goal is to have as straight of a back as possible")
        if a2 < 45:
            print("\tWhen going down for a pushup try to keep your arms from a straight position to a 90 degree angle and make sure they are shoulder width apart")
        if a2 > 60:
            print("\tWhen doing pushups you should go down far enough for your elbows to be bent at a 90 degree angle like and L shape")

    #Plank Feedback
    if task == 2:
        print("Plank Results")
        print("\tYour back average bend was {} degree".format(a1))
        if a1 < 140:
            print("\tTry to keep your back more straight a good way is to keep your but down")
            print("\tThe goal is to have as straight of a back as possible")

    #Situp Feedback
    if task == 3:
        print("Situp Results")
        print("")


In [180]:
def calculate_angle(a, b, c):
    """Calculates angle with 3 given points"""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0/np.pi)

    if angle > 180.0:
        angle = 360-angle

    return angle

In [181]:
def get_marks(task, landmarks):
    """
    Gets landmarks needed for push ups
    task: Task number given to determine what needs to be returned
        1: Pushup
        2: Plank
        3: Situp
        4: Squat
    landmarks: List of seen landmarks
    """
    if task == 1:
        #Pushup Landmarks
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        return shoulder, elbow, wrist, hip, knee
    if task == 2:
        #Plank Landmarks
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        return shoulder, elbow, wrist, hip, knee
    if task == 3:
        #Situp Landmarks
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        return shoulder, hip, knee
    if task == 4:
        #Squat Landmarks
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        head = [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
        return shoulder, head, hip, knee


In [182]:
#cap = cv2.VideoCapture(0)

#Plank test files
#cap = cv2.VideoCapture("TestFiles/Plank1.mp4")
cap = cv2.VideoCapture("TestFiles/Plank2.mp4")

#Pushup test files for presentation
#cap = cv2.VideoCapture("TestFiles/Pushup1.mp4")
#cap = cv2.VideoCapture("TestFiles/Pushup2.mp4")
#cap = cv2.VideoCapture("TestFiles/Pushup3.mp4")

#Tank input for selection screen lator
# Task 1 = Pushup
# Task 2 = Plank
# Task 3 = Situp
# Task 4 = Squat

task = None
task = int(input("What task are you doing?"))
counter = 0
back_average = []
stage = None
a2min = 181

#Recording
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('results/latest.avi', fourcc, 20.0, (640,480))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Checks for video end
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

        # plots landmarks for specific tasks
            if task == 1 or task == 2:
                # pushups and plank
                shoulder, elbow, wrist, hip, knee = get_marks(task, landmarks)

                angle = calculate_angle(shoulder, elbow, wrist)
                angle2 = calculate_angle(shoulder, hip, knee)
                cv2.putText(image, str(round(angle)),
                            tuple(np.multiply(elbow, [640, 700]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                    )
                cv2.putText(image, str(round(angle2)),
                            tuple(np.multiply(hip, [1000, 480]).astype(int)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA
                                    )
                if task == 1:
                    if angle < a2min:
                        a2min = round(angle)
                    back_average.append(angle2)
                    if angle > 160 and stage == "down":
                        stage = "up"
                        counter += 1
                    if angle < 60:
                        stage = "down"
                else:
                    back_average.append(angle2)

        except:
            pass

        # text and container setup
        if task == 1:
            # pushup Counter
            cv2.rectangle(image, (0,0), (225,73), (255,0,255), -1)

            cv2.putText(image, 'Count', (15,12),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), (10,60),
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 1, cv2.LINE_AA)
            name = "Pushup"
        if task == 2:
            name = "Plank"

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=2))


        cv2.imshow(name, image)

        out.write(image)

        

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    if task == 1 or task == 2:
        for i in range(len(back_average)):
            if back_average[i- 1] < 90:
                back_average.pop(i - 1)
            back_avg = round(sum(back_average) / len(back_average))
        feedback(task, back_avg, a2min, counter)

Plank Results
	Your back average bend was 116 degree
	Try to keep your back more straight a good way is to keep your but down
	The goal is to have as straight of a back as possible
